In [1]:
from json import load

In [2]:
with open("../data/example_build.json","r") as build_file:
    build_config = load(build_file)
    
inputs = []
train_config = {
    "dataset":None,
    "optimizer":None,
    
    "model":None,
    "compile":None
}


for _id,config in build_config.items():
    if config['type'] == 'Input':
        inputs.append(_id)
        
    elif config['type'] == 'Dataset':
        train_config['dataset'] = _id
        
    elif config['type'] == 'Optimizer':
        train_config['optimizer'] = _id
    
build_config['input_nodes'] = inputs
levels = [ set() for i in range(len(build_config))]
def setLevel(node,config,di=0):
    levels[di].add(node)
    if build_config[node]['connections']['outbound']:
        for next_node in build_config[node]['connections']['outbound']: 
            setLevel(next_node,build_config,di+1)


for inp in inputs:
    setLevel(inp,build_config,0)
    
build_config['levels'] = levels
    
levels = [list(level) for level in levels if len(level)]
levels

[['input_1'],
 ['dense_1'],
 ['batchnormalization_1'],
 ['activation_1'],
 ['model_1'],
 ['compile_1'],
 ['train_1']]

In [21]:
def build_inbound(inbound:list)->str:
    return ( "([" + ", ".join(inbound) + "])" if len(inbound) > 1 else "(" + inbound[0] + ")" ) if len(inbound) else ""

def set_argument(argument,config):
    value = config['value']
    try:
        value = eval(value)
    except:
        pass
    value = ( None if value == 'None' else f"'{value}'" ) if config['type'] == 'text' else value
    return f"    {argument}={value.__repr__()},"
    
def build_arguments(arguments:dict)->str:
    arguments = '\n'.join([ set_argument(arg,cnf) for arg,cnf in arguments.items() ])
    return arguments

def build_input(layer,build_config,*args,**kwargs)->str:
    arguments =  build_arguments(layer['arguments'])
    return f"""{layer['id']} = layers.Input(
{arguments}
)"""

def build_default(layer,build_config,*args,**kwargs)->str:
    arguments =  build_arguments(layer['arguments'])
    inbound = build_inbound(layer['connections']['inbound'])
    
    return f"""{layer['id']} = layers.{layer['type']}(
{arguments}
){inbound}"""

def build_model(layer,build_config,*args,**kwargs)->str:
    train_config['model'] = layer
    return f"""{layer['id']} = keras.Model(
    [ {', '.join(build_config['input_nodes'])}, ],
    [ {', '.join(layer['connections']['inbound'])}, ]
)"""

def build_compile(layer,build_config,*args,**kwargs)->str:
    train_config['compile'] = layer
    model,*_ = [node for node in layer['connections']['inbound'] if "model" in node]
    return f"""{model}.compile(
    optimizer={train_config['optimizer'] if train_config['optimizer'] else "'"+layer['arguments']['optmizer']['value']+"'"},
    loss='{layer['arguments']['loss']['value']}'
)"""

def build_train(layer,build_config,*args,**kwargs)->str:
    return ""

build_functions = {
    "Input":build_input,
    "default":build_default,
    "Model":build_model,
    "Compile":build_compile,
    "Train":build_train
}

build = ''

for level in levels:
    for layer in level:
        layer = build_config[layer]
        if layer['type'] in build_functions: 
            build += build_functions[layer['type']](layer,build_config) + '\n\n'
        else:
            build += build_functions['default'](layer,build_config) + '\n\n'

In [22]:
code = """#-*- Code generated by Tensorflow GUI -*-
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers,optimizers,losses,metrics

{dataset}
dataset = Dataset()

{build}
""".format(
    dataset=build_config[train_config['dataset']]['arguments']['dataset']['value'][225:],
    build=build
)

print (code)

#-*- Code generated by Tensorflow GUI -*-
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers,optimizers,losses,metrics

#<dataset id=mnist_1>
class Dataset:
    """
    Dataset will be used in training 

    The dataset object needs to have following attributes

    train_x : np.ndarray -> Training features
    train_y : np.ndarray -> Training labels 
    test_x : np.ndarray -> Testing features
    test_y : np.ndarray -> Testing labels

    validate : bool -> Weather use validation data or not

    batch_size : int -> Batch size
    epochs : int -> Number of epochs
    batches : int -> Number of batches ( Will be calculated automatically )
    """
    train_x = None
    test_x = None
    train_y = None
    test_y = None

    validate = True

    def __init__(self) -> None:
        """
        Load dataset and set required variables.
        """

        (X,Y),(x,y) = keras.datasets.mnist.load_data()

        